In [ ]:
from __future__ import annotations

from pathlib import Path
from dataclasses import dataclass
from typing import List, Iterable, Tuple, Dict, Optional, Callable, Any
from heapq import nsmallest

import pandas as pd
import numpy as np
import itertools
import random
import os, time, json
import torch

from google.colab import drive

from cayleypy_pancake.utils.permutation import parse_permutation, apply_move_copy, apply_moves
from cayleypy_pancake.utils.moves import moves_to_str, moves_len
from cayleypy_pancake.utils.io import ensure_dir
from cayleypy_pancake.utils.time import now_str
from cayleypy_pancake.utils.seed import set_seed
from cayleypy_pancake.utils.logging import log_print

from cayleypy_pancake.search.baseline import pancake_sort_moves
from cayleypy_pancake.search.heuristics import breakpoints2, is_solved, gap_h, mix_h, make_h
from cayleypy_pancake.search.beam import beam_improve_or_baseline_h


drive.mount("/content/drive")
OUT_DIR = Path("/content/drive/MyDrive/pancake_runs")
OUT_DIR.mkdir(exist_ok=True)

PROGRESS_PATH = os.path.join(OUT_DIR, "submission_progress.csv")
FINAL_PATH = os.path.join(OUT_DIR, "submission.csv")
BASELINE_PATH = os.path.join(OUT_DIR, "baseline_submission.csv")
TEST_PATH = os.path.join(OUT_DIR, "test.csv")

device = "cuda" if torch.cuda.is_available() else "cpu"

if os.path.exists(PROGRESS_PATH):
    prog_df = pd.read_csv(PROGRESS_PATH)
    progress_map = dict(zip(prog_df["id"].astype(int).values, prog_df["solution"].values))
    print("Resume progress:", len(progress_map))
else:
    progress_map = {}
    print("No progress, start fresh.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No progress, start fresh.


Данный набор функций реализует базовые утилиты для работы с перестановками и решениями задачи pancake sorting.

Функции parse_permutation, moves_to_str и moves_len отвечают за безопасное преобразование данных между форматами. Это позволяет единообразно работать с результатами классических алгоритмов, эвристик и ML-моделей в одном пайплайне.

Функция pancake_sort_moves реализует классический жадный алгоритм pancake sorting и используется как корректный базовый ориентир. Алгоритм работает за O(n^2) и гарантированно гарантированно строит допустимую последовательность префиксных разворотов для любой перестановки и служит эталоном для проверки корректности, оценки качества и сравнения более сложных методов.

In [ ]:
def parse_permutation(raw: str) -> List[int]:
    if raw is None:
        return []
    raw = str(raw).strip()
    if raw == "":
        return []
    return [int(tok) for tok in raw.split(",") if tok.strip() != ""]

def moves_to_str(moves: List[int]) -> str:
    return ".".join(f"R{k}" for k in moves)

def moves_len(sol) -> int:
    if sol is None or (isinstance(sol, float) and pd.isna(sol)):
        return 0
    s = str(sol).strip()
    if s == "":
        return 0
    return s.count(".") + 1

def pancake_sort_moves(perm: Iterable[int]) -> List[int]:
    a = list(perm)
    n = len(a)
    if n <= 1:
        return []

    pos = [0] * n
    for i, v in enumerate(a):
        pos[v] = i

    moves: List[int] = []

    def do_flip(k: int) -> None:
        if k <= 1:
            return
        i, j = 0, k - 1
        while i < j:
            vi, vj = a[i], a[j]
            a[i], a[j] = vj, vi
            pos[vi], pos[vj] = j, i
            i += 1
            j -= 1

    for target in range(n - 1, 0, -1):
        idx = pos[target]
        if idx == target:
            continue
        if idx != 0:
            do_flip(idx + 1)
            moves.append(idx + 1)
        do_flip(target + 1)
        moves.append(target + 1)

    return moves

Этот блок подготавливает тестовый датасет и извлекает из него базовую структурную информацию — размер перестановки n. Мы явно восстанавливаем длину каждой перестановки из строкового представления, а затем считаем распределение по n, чтобы понимать, с какими размерами входов мы работаем. Это важно для анализа покрытия теста, стратификации результатов и последующего сравнения моделей по сложности входа.

In [ ]:
test_df = pd.read_csv(TEST_PATH)

test_df["n"] = test_df["permutation"].apply(
    lambda x: len(parse_permutation(x))
)

n_counts = (
    test_df["n"]
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={"index": "n"})
)

n_counts

,n,count
0,5,5
1,12,200
2,15,200
3,16,200
4,20,200
5,25,200
6,30,200
7,35,200
8,40,200
9,45,200


# Heuristic without DL

Этот набор функций реализует эвристики качества состояния для pancake-задачи, быстрые примитивы применения ходов и проверки решения, улучшатель базового решения через beam search с отсечениями, и экспериментальный грид-поиск по гиперпараметрам эвристики и beam search.

Основная идея: берём базовое решение (по умолчанию классический pancake sort), используем его длину как верхнюю границу и пытаемся найти более короткий путь, исследуя пространство состояний ограниченным лучом (beam) и эвристическим приоритетом. Затем прогоняем это по множеству тест-кейсов и конфигураций, сохраняя метрики (gain, ok, время) для анализа.

breakpoints2 считает число «разрывов» в перестановке: соседние элементы, которые не идут подряд по значению (разность по модулю не равна 1). Дополнительно учитывается штраф, если первый элемент не равен 0, что усиливает давление на «правильный старт». Эта метрика используется как простая эвристика «насколько мы далеко от упорядоченного вида».

gap_h считает разрывы в последовательности, включая виртуальные границы -1 слева и n справа. Это стандартная идея для pancake-задачи: корректный порядок соответствует цепочке -1,0,1,...,n, и каждое нарушение смежности увеличивает оценку. В отличие от breakpoints2, здесь учитываются также граничные условия.

mix_h комбинирует две эвристики: gap_h как основную и breakpoints2 как добавочный штраф с весом alpha. Такая смесь позволяет регулировать «жёсткость» эвристики, балансируя чувствительность к локальным разрывам и предпочтение правильного префикса. Возвращается вещественное значение, чтобы удобно масштабировать вклад.

beam_improve_or_baseline_h пытается улучшить базовое решение ограниченным поиском (beam search) с эвристической оценкой f = g + w*h. Длина базового решения используется как текущая верхняя граница: все пути, которые уже не могут быть лучше, отсекаются. Если улучшение найдено, возвращается более короткий путь, иначе — исходный baseline.

Функция выполняет пакетный прогон экспериментов: на наборе кейсов перебирает все комбинации гиперпараметров (alpha, w, beam_width, depth). Для каждого запуска строится решение улучшателем, проверяется корректность и считаются метрики качества относительно baseline. Результаты собираются в таблицу для последующего анализа и выбора конфигурации.

In [ ]:
def breakpoints2(state: List[int]) -> int:
    n = len(state)
    b = 0
    for i in range(n - 1):
        if abs(state[i] - state[i + 1]) != 1:
            b += 1
    if n > 0 and state[0] != 0:
        b += 1
    return b

def is_solved(state: List[int]) -> bool:
    return all(v == i for i, v in enumerate(state))

def apply_move_copy(state: List[int], k: int) -> List[int]:
    nxt = state[:]
    nxt[:k] = reversed(nxt[:k])
    return nxt

def apply_moves(perm: List[int], moves: List[int]) -> List[int]:
    a = perm[:]
    for k in moves:
        a[:k] = reversed(a[:k])
    return a

def gap_h(state: List[int]) -> int:
    n = len(state)
    prev = -1
    gaps = 0
    for x in state:
        if abs(x - prev) != 1:
            gaps += 1
        prev = x
    if abs(n - prev) != 1:
        gaps += 1
    return gaps

def mix_h(state: List[int], alpha: float = 0.5) -> float:
    return gap_h(state) + alpha * breakpoints2(state)

def make_h(alpha: float) -> Callable[[List[int]], float]:
    if alpha == 0.0:
        return lambda s: float(gap_h(s))
    return lambda s: float(mix_h(s, alpha=alpha))


def beam_improve_or_baseline_h(
    perm: Iterable[int],
    *,
    baseline_moves_fn: Callable[[Iterable[int]], List[int]],
    h_fn: Callable[[List[int]], float],
    beam_width: int = 8,
    depth: int = 12,
    w: float = 1.0,
    log: bool = False,
    log_every_layer: int = 1,
) -> List[int]:
    start = list(perm)

    base_moves = baseline_moves_fn(start)
    best_len = len(base_moves)
    if best_len <= 1:
        return base_moves

    apply_move = apply_move_copy
    solved = is_solved
    h_local = h_fn
    w_local = w
    k_values = range(2, len(start) + 1)

    beam: List[Tuple[float, int, List[int], List[int]]] = [
        (w_local * float(h_local(start)), 0, start, [])
    ]

    best_path: Optional[List[int]] = None
    best_g: Dict[Tuple[int, ...], int] = {tuple(start): 0}

    log_print(log, f"[beam] start n={len(start)} base_len={best_len} bw={beam_width} depth={depth} w={w}")

    for layer in range(1, depth + 1):
        candidates: List[Tuple[float, int, List[int], List[int]]] = []
        improved_this_layer = 0
        for f, g, state, path in beam:
            if g >= best_len:
                continue

            for k in k_values:
                new_g = g + 1
                if new_g >= best_len:
                    continue

                nxt = apply_move(state, k)
                key = tuple(nxt)

                prevg = best_g.get(key)
                if prevg is not None and prevg <= new_g:
                    continue
                best_g[key] = new_g

                if solved(nxt):
                    best_len = new_g
                    best_path = path + [k]
                    improved_this_layer += 1
                    continue

                h = float(h_local(nxt))
                new_f = new_g + w_local * h
                if new_f < best_len:
                    candidates.append((new_f, new_g, nxt, path + [k]))

        if log and (layer % max(1, log_every_layer) == 0):
            log_print(
                True,
                f"[beam] layer={layer:03d} beam_in={len(beam)} cand={len(candidates)} "
                f"improved={improved_this_layer} best_len={best_len}"
            )

        if not candidates:
            break

        beam = nsmallest(beam_width, candidates, key=lambda x: x[0])
        if best_len <= 2:
            break

    return best_path if best_path is not None else base_moves

def select_cases_per_n(
    df: pd.DataFrame,
    n_list: List[int],
    k: int = 5,
    seed: int = 42,
) -> pd.DataFrame:
    rng = random.Random(seed)
    rows = []
    for n in n_list:
        sub = df[df["n"] == n]
        if len(sub) < k:
            raise ValueError(f"Not enough samples for n={n}: have {len(sub)}, need {k}")
        idxs = list(sub.index)
        rng.shuffle(idxs)
        chosen = idxs[:k]
        rows.append(df.loc[chosen])
    return pd.concat(rows, axis=0).reset_index(drop=True)

@dataclass
class RunRow:
    id: int
    n: int
    base_len: int
    alpha: float
    w: float
    beam_width: int
    depth: int
    ok: bool
    steps: int
    gain: int
    time_sec: float


def log_print(enabled: bool, msg: str) -> None:
    if enabled:
        print(msg, flush=True)

def run_grid(
    mini_df: pd.DataFrame,
    *,
    alphas: List[float],
    ws: List[float],
    beam_widths: List[int],
    depths: List[int],
    baseline_moves_fn: Callable[[Iterable[int]], List[int]] = pancake_sort_moves,
    log: bool = True,
    log_each: int = 1,
    beam_log: bool = False,
    beam_log_every_layer: int = 5,
) -> pd.DataFrame:
    rows: List[dict] = []
    total_cfg = len(alphas) * len(ws) * len(beam_widths) * len(depths)
    total_cases = len(mini_df)
    total_runs = total_cfg * total_cases

    log_print(log, f"[grid] cases={total_cases} cfg_per_case={total_cfg} total_runs={total_runs}")
    parsed: List[Tuple[int, int, List[int], int]] = []
    for i in range(total_cases):
        row = mini_df.iloc[i]
        perm = parse_permutation(row.permutation)
        n = len(perm)
        base_len = len(baseline_moves_fn(perm))
        parsed.append((int(row.id), n, perm, base_len))

    run_idx = 0
    for case_i, (rid, n, perm, base_len) in enumerate(parsed):
        log_print(log, f"\n[case {case_i+1}/{total_cases}] id={rid} n={n} base_len={base_len}")

        for cfg_i, (alpha, w, bw, d) in enumerate(itertools.product(alphas, ws, beam_widths, depths), start=1):
            run_idx += 1
            do_cfg_log = log and (cfg_i % max(1, log_each) == 0)

            t0 = time.time()
            h_fn = make_h(alpha)
            sol = beam_improve_or_baseline_h(
                perm,
                baseline_moves_fn=baseline_moves_fn,
                h_fn=h_fn,
                beam_width=bw,
                depth=d,
                w=w,
                log=(beam_log and do_cfg_log),
                log_every_layer=beam_log_every_layer,
            )
            dt = time.time() - t0

            ok = (apply_moves(perm, sol) == list(range(n)))
            steps = len(sol)
            gain = base_len - steps

            if do_cfg_log:
                log_print(
                    True,
                    f"[run {run_idx}/{total_runs}] cfg={cfg_i:03d}/{total_cfg} "
                    f"alpha={alpha} w={w} bw={bw} depth={d} -> steps={steps} gain={gain} ok={ok} t={dt:.3f}s"
                )

            rows.append({
                "id": rid,
                "n": n,
                "base_len": base_len,
                "alpha": alpha,
                "w": w,
                "beam_width": bw,
                "depth": d,
                "ok": ok,
                "steps": steps,
                "gain": gain,
                "time_sec": dt,
            })

    df = pd.DataFrame(rows)
    return df

Этот блок формирует репрезентативный поднабор тестовых случаев по размерам перестановок и запускает полный грид-эксперимент по улучшению базовых решений. Для каждого выбранного размера n случайно отбираются несколько кейсов, после чего перебираются комбинации параметров эвристики и beam search. Результатом является таблица с метриками качества и времени, пригодная для анализа масштабируемости и подбора оптимальных конфигураций.

In [ ]:
n_list = [12, 15, 16, 20, 25, 30, 35, 40, 45, 50, 75, 100]
mini_df = select_cases_per_n(test_df, n_list, k=5, seed=42)

alphas = [0.0, 0.5, 1.0]
ws = [0.5, 1.0, 1.5]
beam_widths = [64, 128, 256]
depths = [64, 128, 192]

df = run_grid(
    mini_df,
    alphas=alphas,
    ws=ws,
    beam_widths=beam_widths,
    depths=depths,
    log=True,
    log_each=100,
    beam_log=True,
    beam_log_every_layer=5
)

df.head()


[grid] cases=60 cfg_per_case=81 total_runs=4860

[case 1/60] id=71 n=12 base_len=19

[case 2/60] id=192 n=12 base_len=9

[case 3/60] id=106 n=12 base_len=16

[case 4/60] id=198 n=12 base_len=17

[case 5/60] id=116 n=12 base_len=18

[case 6/60] id=281 n=15 base_len=22

[case 7/60] id=299 n=15 base_len=20

[case 8/60] id=357 n=15 base_len=22

[case 9/60] id=368 n=15 base_len=20

[case 10/60] id=333 n=15 base_len=23

[case 11/60] id=527 n=16 base_len=20

[case 12/60] id=480 n=16 base_len=25

[case 13/60] id=459 n=16 base_len=26

[case 14/60] id=564 n=16 base_len=26

[case 15/60] id=600 n=16 base_len=28

[case 16/60] id=707 n=20 base_len=31

[case 17/60] id=792 n=20 base_len=29

[case 18/60] id=699 n=20 base_len=34

[case 19/60] id=660 n=20 base_len=29

[case 20/60] id=736 n=20 base_len=27

[case 21/60] id=870 n=25 base_len=38

[case 22/60] id=961 n=25 base_len=38

[case 23/60] id=948 n=25 base_len=46

[case 24/60] id=933 n=25 base_len=40

[case 25/60] id=903 n=25 base_len=38

[case 26/60]

,id,n,base_len,alpha,w,beam_width,depth,ok,steps,gain,time_sec
0,71,12,19,0.0,0.5,64,64,True,11,8,0.050677
1,71,12,19,0.0,0.5,64,128,True,11,8,0.136621
2,71,12,19,0.0,0.5,64,192,True,11,8,0.036501
3,71,12,19,0.0,0.5,128,64,True,11,8,0.074562
4,71,12,19,0.0,0.5,128,128,True,11,8,0.080702


Во всех конфигурациях beam search успешно находит корректные решения для всех кейсов (solved = 60), а средний выигрыш относительно baseline стабилен и лежит в диапазоне 28-29 шагов. Качество решения растёт с увеличением beam_width, но при этом время выполнения растёт существенно быстрее. Влияние параметров alpha и w на итоговое качество оказывается вторичным. Увеличение глубины поиска (depth) почти не даёт выигрыша, начиная с умеренных значений.

In [ ]:
summary = (
    df.groupby(["alpha","w","beam_width","depth"], as_index=False)
      .agg(
          solved=("ok","sum"),
          mean_gain=("gain","mean"),
          mean_steps=("steps","mean"),
          mean_time=("time_sec","mean"),
      )
      .sort_values(["solved","mean_gain","mean_time"], ascending=[False, False, True])
)

summary.head(20)


,alpha,w,beam_width,depth,solved,mean_gain,mean_steps,mean_time
7,0.0,0.5,256,128,60,28.583333,38.366667,12.573641
8,0.0,0.5,256,192,60,28.583333,38.366667,12.693728
16,0.0,1.0,256,128,60,28.566667,38.383333,12.376708
17,0.0,1.0,256,192,60,28.566667,38.383333,12.467067
34,0.5,0.5,256,128,60,28.550000,38.400000,16.833309
35,0.5,0.5,256,192,60,28.550000,38.400000,16.935335
61,1.0,0.5,256,128,60,28.533333,38.416667,16.524541
62,1.0,0.5,256,192,60,28.533333,38.416667,16.904485
31,0.5,0.5,128,128,60,28.183333,38.766667,7.376665
32,0.5,0.5,128,192,60,28.183333,38.766667,7.489081


Далее мы фиксируем alpha=0.0 (gap-эвристика) и w=0.5 и создаем сабмит из наилучших результатов трех beam search.

1. Лучшее качество: (alpha=0.0, w=0.5, bw=256, depth=128)

2. Максимальные beam_depth и beam_width : (alpha=0.0, w=0.5, bw=256, depth=128) (равны по mean_gain)

3. Быстрый и экономный: (alpha=0.0, w=0.5, bw=128, depth=128)

In [ ]:
import os, time
import pandas as pd

def full_eval_top_cfgs(
    test_df: pd.DataFrame,
    n_list: List[int],
    top_cfgs: List[dict],
    *,
    out_csv_path: str,
    baseline_moves_fn=pancake_sort_moves,
    log: bool = True,
    log_every: int = 50,
) -> pd.DataFrame:

    os.makedirs(os.path.dirname(out_csv_path), exist_ok=True)
    sub = test_df[test_df["n"].isin(n_list)].reset_index(drop=True)

    done = set()
    wrote_header = os.path.exists(out_csv_path)
    if wrote_header:
        try:
            prev = pd.read_csv(out_csv_path, usecols=["id", "cfg_idx"])
            done = set(zip(prev["id"].astype(int).tolist(), prev["cfg_idx"].astype(int).tolist()))
            if log:
                print(f"[resume] found existing file with {len(done)} completed runs", flush=True)
        except Exception as e:
            if log:
                print(f"[resume] could not read existing file safely: {e!r} (will append anyway)", flush=True)

    rows_cache = []

    t_global0 = time.time()
    total_cases = len(sub)
    total_runs = total_cases * len(top_cfgs)

    run_idx = 0
    skipped = 0

    for i in range(total_cases):
        rid = int(sub.loc[i, "id"])
        perm = parse_permutation(sub.loc[i, "permutation"])
        n = len(perm)

        base_moves = baseline_moves_fn(perm)
        base_len = len(base_moves)

        for cfg_j, cfg in enumerate(top_cfgs):
            run_idx += 1

            if (rid, cfg_j) in done:
                skipped += 1
                if log and (run_idx % log_every == 0):
                    elapsed = time.time() - t_global0
                    speed = (run_idx - skipped) / max(1e-9, elapsed)
                    log_print(True, f"[full] {run_idx}/{total_runs} runs | skipped={skipped} | speed={speed:.3f} new_runs/s")
                continue

            alpha = float(cfg["alpha"])
            w = float(cfg["w"])
            bw = int(cfg["beam_width"])
            depth = int(cfg["depth"])

            t0 = time.time()
            status = "ok"
            err_txt = ""

            try:
                h_fn = make_h(alpha)
                moves = beam_improve_or_baseline_h(
                    perm,
                    baseline_moves_fn=baseline_moves_fn,
                    h_fn=h_fn,
                    beam_width=bw,
                    depth=depth,
                    w=w,
                    log=False,
                )

                if apply_moves(perm, moves) != list(range(n)):
                    moves = base_moves
                    status = "fallback_baseline"

            except Exception as e:
                moves = base_moves
                status = "error_fallback_baseline"
                err_txt = repr(e)

            dt = time.time() - t0

            steps = len(moves)
            gain = base_len - steps
            sol_str = moves_to_str(moves)

            row = {
                "id": rid,
                "n": n,
                "cfg_idx": cfg_j,
                "alpha": alpha,
                "w": w,
                "beam_width": bw,
                "depth": depth,
                "base_len": base_len,
                "ok": (status == "ok"),
                "steps": steps,
                "gain": gain,
                "time_sec": dt,
                "solution": sol_str,
                "status": status,
                "error": err_txt,
            }

            rows_cache.append(row)
            done.add((rid, cfg_j))

            if len(rows_cache) >= 200:
                pd.DataFrame(rows_cache).to_csv(
                    out_csv_path,
                    mode="a",
                    header=not wrote_header,
                    index=False
                )
                wrote_header = True
                rows_cache.clear()

            if log and (run_idx % log_every == 0):
                elapsed = time.time() - t_global0
                new_done = run_idx - skipped
                speed = new_done / max(1e-9, elapsed)  # new runs/sec
                log_print(
                    True,
                    f"[full] {run_idx}/{total_runs} runs | new={new_done} skipped={skipped} | "
                    f"n={n} cfg={cfg_j} steps={steps} gain={gain} status={status} | {speed:.3f} new_runs/s"
                )

    if rows_cache:
        pd.DataFrame(rows_cache).to_csv(
            out_csv_path,
            mode="a",
            header=not wrote_header,
            index=False
        )

    return pd.read_csv(out_csv_path)


Выполним полный прогон выбранных конфигураций (top_cfgs) на множестве всех тестовых примеров с размерами из n_list и сохраним результаты в CSV.

In [ ]:
os.makedirs(OUT_DIR, exist_ok=True)

OUT_CSV = f"{OUT_DIR}/full_eval_top_cfgs_seed42.csv"
n_list = [5, 12, 15, 16, 20, 25, 30, 35, 40, 45, 50, 75, 100]

top_cfgs = [
    {"alpha": 0.0, "w": 0.5, "beam_width": 128, "depth": 128},
    {"alpha": 0.0, "w": 0.5, "beam_width": 256, "depth": 128},
    {"alpha": 0.0, "w": 0.5, "beam_width": 256, "depth": 192},
]

df_full = full_eval_top_cfgs(
    test_df, n_list, top_cfgs,
    out_csv_path=OUT_CSV,
    log=True,
    log_every=100,
)

df_full.head()


Эксперименты показывают, что увеличение ширины луча с 128 до 256 даёт лишь маргинальный прирост качества (≈0.27 шага в среднем), при этом увеличивая время выполнения более чем в два раза. Увеличение глубины поиска с 128 до 192 не оказывает измеримого влияния на качество.

Конфигурация beam_width=128, depth=128 выглядит как оптимальный компромисс между качеством и вычислительной стоимостью.

In [ ]:
summary_full = (
    df_full.groupby(["cfg_idx","alpha","w","beam_width","depth"], as_index=False)
           .agg(
               solved=("ok","sum"),
               mean_gain=("gain","mean"),
               mean_steps=("steps","mean"),
               mean_time=("time_sec","mean"),
           )
           .sort_values(["solved","mean_gain","mean_time"], ascending=[False, False, True])
)
summary_full


,cfg_idx,alpha,w,beam_width,depth,solved,mean_gain,mean_steps,mean_time
2,2,0.0,0.5,256,192,2405,27.859875,38.119335,11.731055
1,1,0.0,0.5,256,128,2405,27.859875,38.119335,11.741843
0,0,0.0,0.5,128,128,2405,27.585447,38.393763,4.977610


Функция сравнивает качество решений из submission_df с выбранным baseline-алгоритмом на полном test_df. Для каждого примера она считает длину baseline-решения и длину решения из сабмита, накапливая суммарные значения и статистику (сколько улучшили/ухудшили, максимальный выигрыш и т.п.).

In [ ]:
from typing import Dict, Optional
import time
import pandas as pd

def evaluate_submission_vs_baseline(
    test_df: pd.DataFrame,
    submission_df: pd.DataFrame,
    *,
    baseline_moves_fn,
    log_every: int = 0,
    save_detailed_path: Optional[str] = None,
) -> Dict:
    t0 = time.time()
    sub_map = dict(zip(submission_df["id"].astype(int), submission_df["solution"].astype(str)))

    sum_base = 0
    sum_sub = 0
    improved = same = worse = 0
    total_gain_pos = 0
    max_gain = 0
    max_gain_id = None

    N = len(test_df)

    detailed_rows = [] if save_detailed_path else None

    for i, row in enumerate(test_df.itertuples(index=False), start=1):
        rid = int(row.id)
        perm = parse_permutation(row.permutation)

        base = baseline_moves_fn(perm)
        lb = len(base)

        sol = sub_map.get(rid, "")
        lz = moves_len(sol)

        sum_base += lb
        sum_sub += lz

        gain = lb - lz
        if gain > 0:
            improved += 1
            total_gain_pos += gain
            if gain > max_gain:
                max_gain = gain
                max_gain_id = rid
        elif gain == 0:
            same += 1
        else:
            worse += 1

        if detailed_rows is not None:
            detailed_rows.append({
                "id": rid,
                "n": len(perm),
                "base_len": lb,
                "sub_len": lz,
                "gain": gain,
            })

        if log_every and (i % log_every == 0 or i == 1 or i == N):
            print(f"[{i:4d}/{N}] base={lb:3d} sub={lz:3d} gain={gain:3d}  elapsed={time.time()-t0:7.1f}s",
                  flush=True)

    dt = time.time() - t0

    if save_detailed_path:
        pd.DataFrame(detailed_rows).to_csv(save_detailed_path, index=False)

    return {
        "baseline_total": sum_base,
        "submission_total": sum_sub,
        "total_gain": (sum_base - sum_sub),
        "improved_cases": improved,
        "same_cases": same,
        "worse_cases": worse,
        "avg_gain_when_improved": (total_gain_pos / improved) if improved else 0.0,
        "max_gain": max_gain,
        "max_gain_id": max_gain_id,
        "time_sec": dt,
        "sec_per_sample": dt / max(1, N),
        "mean_baseline_len": sum_base / max(1, N),
        "mean_submission_len": sum_sub / max(1, N),
        "improved_frac": improved / max(1, N),
    }


Мы сравнили итоговый submission с baseline-алгоритмом (классический pancake sorting) на всём тестовом наборе из 2405 перестановок. Submission улучшает baseline в 2401 случаях (99.83%), не ухудшая результат ни в одном кейсе; ещё в 4 случаях длины совпадают. Суммарная длина решений снизилась с 158 680 до **91 594** ходов, что даёт общий выигрыш 67 086 ходов и средний выигрыш 27.94 шага на улучшенных примерах. Максимальный выигрыш на одном кейсе составил 92 шага (id=2342).

In [ ]:
sub_df = pd.read_csv("/content/drive/MyDrive/pancake_runs/submission_bs.csv")
stats = evaluate_submission_vs_baseline(
    test_df=test_df,
    submission_df=sub_df,
    baseline_moves_fn=pancake_sort_moves,
    log_every=50,
    save_detailed_path="sub_vs_base_details.csv",
)
stats

[   1/2405] base=  2 sub=  2 gain=  0  elapsed=    0.0s
[  50/2405] base= 19 sub=  8 gain= 11  elapsed=    0.0s
[ 100/2405] base= 20 sub= 12 gain=  8  elapsed=    0.0s
[ 150/2405] base= 16 sub= 11 gain=  5  elapsed=    0.0s
[ 200/2405] base= 15 sub= 11 gain=  4  elapsed=    0.0s
[ 250/2405] base= 19 sub= 15 gain=  4  elapsed=    0.0s
[ 300/2405] base= 20 sub= 14 gain=  6  elapsed=    0.0s
[ 350/2405] base= 20 sub= 15 gain=  5  elapsed=    0.0s
[ 400/2405] base= 21 sub= 14 gain=  7  elapsed=    0.0s
[ 450/2405] base= 21 sub= 13 gain=  8  elapsed=    0.0s
[ 500/2405] base= 25 sub= 15 gain= 10  elapsed=    0.0s
[ 550/2405] base= 25 sub= 15 gain= 10  elapsed=    0.0s
[ 600/2405] base= 21 sub= 12 gain=  9  elapsed=    0.0s
[ 650/2405] base= 32 sub= 20 gain= 12  elapsed=    0.0s
[ 700/2405] base= 34 sub= 19 gain= 15  elapsed=    0.0s
[ 750/2405] base= 32 sub= 19 gain= 13  elapsed=    0.0s
[ 800/2405] base= 33 sub= 19 gain= 14  elapsed=    0.0s
[ 850/2405] base= 42 sub= 24 gain= 18  elapsed= 

{'baseline_total': 158680,
 'submission_total': 91594,
 'total_gain': 67086,
 'improved_cases': 2401,
 'same_cases': 4,
 'worse_cases': 0,
 'avg_gain_when_improved': 27.9408579758434,
 'max_gain': 92,
 'max_gain_id': 2342,
 'time_sec': 0.2909247875213623,
 'sec_per_sample': 0.00012096664761803007,
 'mean_baseline_len': 65.97920997920998,
 'mean_submission_len': 38.08482328482329,
 'improved_frac': 0.9983367983367983}

Агрегация результатов по размерам перестановок показывает, что выигрыш относительно baseline растёт почти линейно с n. При n=12 средний выигрыш составляет около 5 шагов, тогда как при n=100 он превышает 80 шагов. При этом не наблюдается ни одного случая ухудшения решения. Более того, относительное сокращение длины решения увеличивается с ростом n, что указывает на хорошую масштабируемость предложенного апгрейда и его способность эффективно эксплуатировать структуру pancake-графа на больших размерах.

In [ ]:
details = pd.read_csv("sub_vs_base_details.csv")
(details.groupby("n", as_index=False)
        .agg(mean_gain=("gain","mean"),
             improved=("gain", lambda x: (x>0).sum()),
             worse=("gain", lambda x: (x<0).sum()),
             mean_base=("base_len","mean"),
             mean_sub=("sub_len","mean"))
        .sort_values("n"))

,n,mean_gain,improved,worse,mean_base,mean_sub
0,5,0.200,1,0,3.400,3.200
1,12,4.990,200,0,15.820,10.830
2,15,7.390,200,0,21.130,13.740
3,16,7.965,200,0,22.780,14.815
4,20,11.550,200,0,30.460,18.910
5,25,15.740,200,0,39.615,23.875
6,30,20.625,200,0,49.550,28.925
7,35,24.745,200,0,58.750,34.005
8,40,29.235,200,0,68.470,39.235
9,45,32.975,200,0,77.450,44.475


Классический pancake sorting быстро становится неоптимальным при росте n, тогда как ограниченный эвристический поиск находит всё более короткие траектории, причём выигрыш растёт пропорционально размеру задачи.

# ML-heuristic with DL

In [ ]:
!pip install git+https://github.com/cayleypy/cayleypy.git@e1518b6 --no-deps

  Cloning https://github.com/cayleypy/cayleypy.git (to revision e1518b6) to /tmp/pip-req-build-rxi6fpu1
  Running command git clone --filter=blob:none --quiet https://github.com/cayleypy/cayleypy.git /tmp/pip-req-build-rxi6fpu1
  Running command git checkout -q e1518b6
  Resolved https://github.com/cayleypy/cayleypy.git to commit e1518b6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cayleypy: filename=cayleypy-0.1.0-py3-none-any.whl size=592103 sha256=ba6530e7983b2908468466a2ec1fce46ee6cd69e33c8f571d908835d23ed90dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-cf_l8bti/wheels/bb/02/6c/551e0b1e957367d2581e24443666bf3222295029a6821a4726
Successfully built cayleypy


In [ ]:
import os, time, json, random
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from heapq import nsmallest
from typing import Any, Optional, List, Tuple, Dict, Iterable
from copy import deepcopy
from itertools import product
from cayleypy import PermutationGroups, CayleyGraph

Далее рассмотрим семейство регрессионных моделей.
Реализованы три варианта представления входа:
1. One-hot MLP (простая базовая)
2. One-hot MLP с residual-стеком
3. Embedding-версия

Embedding-регрессор заменяет one-hot представление на обучаемые эмбеддинги токенов (значений перестановки), что обычно существенно экономит память и ускоряет вычисления при больших n. Дополнительно может добавляться позиционный эмбеддинг, позволяющий модели различать одинаковые значения в разных позициях и лучше реагировать на структуру перестановки. После эмбеддингов последовательность разворачивается в вектор и обрабатывается MLP (как в Pilgrim), включая опциональные residual-блоки.

 Функция get_model выбирает архитектуру по cfg["model_type"], позволяя переключать модели без изменения остального пайплайна.

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, hidden_dim: int, dropout_rate: float = 0.1):
        super().__init__()
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.drop = nn.Dropout(dropout_rate)

    def forward(self, x):
        r = x
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = x + r
        x = F.relu(x)
        return x


class Pilgrim(nn.Module):
    """
    One-hot MLP + optional 2nd layer + residual stack.
    Input: z (B,n) int permutation
    """
    def __init__(self, cfg: Dict[str, Any]):
        super().__init__()
        self.dtype = torch.float32
        self.state_size = int(cfg["state_size"])
        self.num_classes = int(cfg["num_classes"])
        self.hd1 = int(cfg.get("hd1", 512))
        self.hd2 = int(cfg.get("hd2", 256))
        self.nrd = int(cfg.get("nrd", 0))
        self.dropout_rate = float(cfg.get("dropout_rate", 0.1))
        self.z_add = 0

        in_dim = self.state_size * self.num_classes
        self.input_layer = nn.Linear(in_dim, self.hd1)
        self.bn1 = nn.BatchNorm1d(self.hd1)
        self.drop1 = nn.Dropout(self.dropout_rate)

        if self.hd2 > 0:
            self.hidden_layer = nn.Linear(self.hd1, self.hd2)
            self.bn2 = nn.BatchNorm1d(self.hd2)
            self.drop2 = nn.Dropout(self.dropout_rate)
            hid = self.hd2
        else:
            self.hidden_layer = None
            self.bn2 = None
            self.drop2 = None
            hid = self.hd1

        self.residual_blocks = None
        if self.nrd > 0:
            self.residual_blocks = nn.ModuleList([ResidualBlock(hid, self.dropout_rate) for _ in range(self.nrd)])

        self.out = nn.Linear(hid, 1)

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        x = F.one_hot((z.long() + self.z_add), num_classes=self.num_classes).view(z.size(0), -1).to(self.dtype)
        x = self.input_layer(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.drop1(x)

        if self.hidden_layer is not None:
            x = self.hidden_layer(x)
            x = self.bn2(x)
            x = F.relu(x)
            x = self.drop2(x)

        if self.residual_blocks is not None:
            for blk in self.residual_blocks:
                x = blk(x)

        return self.out(x).flatten()


class SimpleMLP(nn.Module):
    """
    Configurable one-hot MLP.
    """
    def __init__(self, cfg: Dict[str, Any]):
        super().__init__()
        self.dtype = torch.float32
        self.state_size = int(cfg["state_size"])
        self.num_classes = int(cfg["num_classes"])
        self.z_add = 0

        layers = list(cfg["layers"])
        batch_norms = list(cfg.get("batch_norms", [True]*len(layers)))
        dropouts = cfg.get("dropout_rates", 0.1)
        activations = cfg.get("activations", nn.ReLU())

        if not isinstance(dropouts, list):
            dropouts = [dropouts]*len(layers)
        if not isinstance(activations, list):
            activations = [activations]*len(layers)

        in_dim = self.state_size * self.num_classes
        seq = []
        for h, bn, act, dr in zip(layers, batch_norms, activations, dropouts):
            seq.append(nn.Linear(in_dim, int(h)))
            if bn:
                seq.append(nn.BatchNorm1d(int(h)))
            seq.append(act)
            seq.append(nn.Dropout(float(dr)))
            in_dim = int(h)
        seq.append(nn.Linear(in_dim, 1))
        self.net = nn.Sequential(*seq)

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        x = F.one_hot((z.long() + self.z_add), num_classes=self.num_classes).view(z.size(0), -1).to(self.dtype)
        return self.net(x).flatten()


class EmbMLP(nn.Module):
    """
    Embedding-based regressor (recommended).
    Input: z (B,n) ints in [0..n-1]
    """
    def __init__(self, cfg: Dict[str, Any]):
        super().__init__()
        self.dtype = torch.float32
        self.state_size = int(cfg["state_size"])
        self.num_classes = int(cfg["num_classes"])
        assert self.state_size == self.num_classes, "For pancakes, state_size==num_classes==n"
        self.z_add = 0

        d = int(cfg.get("emb_dim", 32))
        self.use_pos_emb = bool(cfg.get("use_pos_emb", True))
        dropout = float(cfg.get("dropout_rate", 0.1))

        self.token_emb = nn.Embedding(self.num_classes, d)
        if self.use_pos_emb:
            self.pos_emb = nn.Embedding(self.state_size, d)
            self.register_buffer("_pos_idx", torch.arange(self.state_size, dtype=torch.long), persistent=False)

        hd1 = int(cfg.get("hd1", 512))
        hd2 = int(cfg.get("hd2", 256))
        nrd = int(cfg.get("nrd", 0))

        in_dim = self.state_size * d
        self.fc1 = nn.Linear(in_dim, hd1)
        self.bn1 = nn.BatchNorm1d(hd1)
        self.drop1 = nn.Dropout(dropout)

        if hd2 > 0:
            self.fc2 = nn.Linear(hd1, hd2)
            self.bn2 = nn.BatchNorm1d(hd2)
            self.drop2 = nn.Dropout(dropout)
            hid = hd2
        else:
            self.fc2 = None
            self.bn2 = None
            self.drop2 = None
            hid = hd1

        self.residual_blocks = None
        if nrd > 0:
            self.residual_blocks = nn.ModuleList([ResidualBlock(hid, dropout) for _ in range(nrd)])

        self.out = nn.Linear(hid, 1)

        nn.init.normal_(self.token_emb.weight, mean=0.0, std=0.02)
        if self.use_pos_emb:
            nn.init.normal_(self.pos_emb.weight, mean=0.0, std=0.02)

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        z = (z.long() + self.z_add).clamp(min=0, max=self.num_classes-1)
        x = self.token_emb(z)  # (B,n,d)
        if self.use_pos_emb:
            pos = self._pos_idx.to(z.device)
            x = x + self.pos_emb(pos)[None, :, :]
        x = x.reshape(z.size(0), -1).to(self.dtype)

        x = self.fc1(x); x = self.bn1(x); x = F.relu(x); x = self.drop1(x)
        if self.fc2 is not None:
            x = self.fc2(x); x = self.bn2(x); x = F.relu(x); x = self.drop2(x)
        if self.residual_blocks is not None:
            for blk in self.residual_blocks:
                x = blk(x)
        return self.out(x).flatten()


def get_model(cfg: Dict[str, Any]) -> nn.Module:
    mt = cfg.get("model_type", "EmbMLP")
    if mt == "EmbMLP":
        return EmbMLP(cfg)
    if mt == "MLPRes1":
        return Pilgrim(cfg)
    if mt == "MLP":
        return SimpleMLP(cfg)
    raise ValueError(f"Unknown model_type={mt}")


Зададим вспомогательные утилиты для воспроизводимых экспериментов и удобной организации артефактов. Здесь фиксируется базовая конфигурация обучения (BASE_CFG), предоставляется способ порождать конфиги с переопределениями (make_cfg) и строится сетка архитектурных вариантов для перебора (build_model_grid). Дополнительно формируется читаемое имя эксперимента (exp_name_from), чтобы результаты можно было систематизировать по параметрам модели.

In [ ]:
def ensure_dir(path: str):
    if path and not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

def now_str():
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

def set_seed(seed: int = 123):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def exp_name_from(cfg: dict, *, n: int, w_gap: float, gap_mode: str):
    mt = cfg.get("model_type", "EmbMLP")
    nrd = int(cfg.get("nrd", 0))
    if mt == "EmbMLP":
        ed = int(cfg.get("emb_dim", -1))
        pos = 1 if bool(cfg.get("use_pos_emb", True)) else 0
        return f"n{n}_{mt}_ed{ed}_pos{pos}_nrd{nrd}_wg{w_gap}_{gap_mode}"
    if mt == "MLPRes1":
        return f"n{n}_{mt}_nrd{nrd}_wg{w_gap}_{gap_mode}"
    return f"n{n}_{mt}_wg{w_gap}_{gap_mode}"

BASE_CFG = dict(
    rw_width=3000,
    rw_mode="mix",
    mix_bfs_frac=0.3,
    nbt_history_depth=1,
    k=4,
    lr=1e-4,
    weight_decay=1e-2,
    batch_size=1024,
    val_ratio=0.15,
    num_epochs=30,
    y_transform="log1p",
    grad_clip=1.0,
    loss_beta=1.0,

    stratify_clip=60,
    stratify_bin_size=1.0,
    sanity_width=2000,
    h_batch_size=8192,
    eval_log_every=10,

    seed=123,
    rw_length_add=30,

    early_stop_patience=6,
    early_stop_min_delta=1e-4,
    early_stop_warmup=3,
    early_stop_restore_best=True,
)

def make_cfg(**overrides):
    cfg = deepcopy(BASE_CFG)
    cfg.update(overrides)
    return cfg

def build_model_grid(
    *,
    emb_dims=(32,64),
    pos_opts=(True, False),
    nrds=(0,2,6),
    hd1=512, hd2=256,
    dropout_rate=0.1,
    include_mlpres1=True,
):
    grid = []

    for ed, pos, nrd in product(emb_dims, pos_opts, nrds):
        grid.append(make_cfg(
            model_type="EmbMLP",
            emb_dim=int(ed),
            use_pos_emb=bool(pos),
            hd1=hd1, hd2=hd2, nrd=int(nrd),
            dropout_rate=float(dropout_rate),
        ))

    if include_mlpres1:
        for nrd in nrds:
            grid.append(make_cfg(
                model_type="MLPRes1",
                hd1=hd1, hd2=hd2, nrd=int(nrd),
                dropout_rate=float(dropout_rate),
            ))
    return grid


Реализуем обучение регрессионной модели на данных, сгенерированных из pancake-графа через случайные обходы (random walks). На каждой эпохе заново генерируется датасет состояний X и целевых значений y, применяется трансформация таргета (например, log1p), после чего данные перемешиваются и делятся на train/val.

Обучение идёт на GPU с AdamW, SmoothL1Loss, клиппингом градиента и CosineAnnealingLR. Sanity-check оценивает корреляцию между предсказаниями модели и лог-трансформированным таргетом, сгенерированным в BFS-режиме. Это быстрый индикатор того, что модель “видит сигнал” и не выдаёт константу или шум. Использование @torch.no_grad() делает функцию дешёвой и безопасной для запуска между эпохами/экспериментами.

In [ ]:
def _y_transform_torch(y: torch.Tensor, mode: str) -> torch.Tensor:
    y = y.detach().float().view(-1)
    if mode is None or mode == "none":
        return y
    if mode == "log1p":
        return torch.log1p(y)
    if mode == "norm_max":
        return y / y.max().clamp_min(1.0)
    raise ValueError(f"Unknown y_transform={mode}")

def _make_bins_np(y: torch.Tensor, clip: int = 60, bin_size: float = 1.0) -> np.ndarray:
    y_np = y.detach().float().cpu().numpy()
    b = np.floor(y_np / float(bin_size)).astype(np.int32)
    b = np.clip(b, 0, int(clip))
    return b

def _gen_walks(cfg, graph, mode: str):
    nbt_hist = int(cfg.get("nbt_history_depth", cfg.get("history_depth", 1)))
    return graph.random_walks(
        width=int(cfg["rw_width"]),
        length=int(cfg["rw_length"]),
        mode=mode,
        nbt_history_depth=nbt_hist,
    )

def train_model_gpu(cfg, model, graph) -> None:
    device = graph.device
    model.to(device)

    bs = int(cfg["batch_size"])
    val_ratio = float(cfg["val_ratio"])
    epochs = int(cfg["num_epochs"])

    y_mode = cfg.get("y_transform", "log1p")
    grad_clip = float(cfg.get("grad_clip", 1.0))
    weight_decay = float(cfg.get("weight_decay", 1e-2))
    loss_beta = float(cfg.get("loss_beta", 1.0))

    strat_clip = int(cfg.get("stratify_clip", 60))
    strat_bin_size = float(cfg.get("stratify_bin_size", 1.0))

    rw_mode = cfg.get("rw_mode", "nbt")
    mix_bfs_frac = float(cfg.get("mix_bfs_frac", 0.3))
    es_patience = int(cfg.get("early_stop_patience", 0))
    es_min_delta = float(cfg.get("early_stop_min_delta", 0.0))
    es_warmup = int(cfg.get("early_stop_warmup", 0))
    es_restore = bool(cfg.get("early_stop_restore_best", True))

    best_val = float("inf")
    best_state = None
    bad_epochs = 0

    opt = torch.optim.AdamW(model.parameters(), lr=float(cfg["lr"]), weight_decay=weight_decay)
    loss_fn = torch.nn.SmoothL1Loss(beta=loss_beta)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)

    for epoch in range(epochs):
        model.train()

        if rw_mode == "mix":
            w_total = int(cfg["rw_width"])
            w_bfs = max(1, int(round(w_total * mix_bfs_frac)))
            w_nbt = max(1, w_total - w_bfs)

            cfg_bfs = dict(cfg); cfg_bfs["rw_width"] = w_bfs
            cfg_nbt = dict(cfg); cfg_nbt["rw_width"] = w_nbt

            X1, y1 = _gen_walks(cfg_bfs, graph, "bfs")
            X2, y2 = _gen_walks(cfg_nbt, graph, "nbt")

            X = torch.cat([X1, X2], dim=0)
            y = torch.cat([y1, y2], dim=0)
        else:
            X, y = _gen_walks(cfg, graph, rw_mode)

        X = X.long()
        y = y.view(-1)
        y_t = _y_transform_torch(y, y_mode)

        M = X.size(0)
        perm_idx = torch.randperm(M, device=device)
        X = X[perm_idx]
        y_t = y_t[perm_idx]

        try:
            bins = _make_bins_np(y_t, clip=strat_clip, bin_size=strat_bin_size)
            from sklearn.model_selection import train_test_split
            idx = np.arange(M)
            idx_tr, idx_va = train_test_split(
                idx, test_size=val_ratio, stratify=bins, shuffle=True, random_state=123
            )
        except Exception:
            val_M = int(M * val_ratio)
            idx_va = np.arange(val_M)
            idx_tr = np.arange(val_M, M)

        idx_tr_t = torch.as_tensor(idx_tr, device=device, dtype=torch.long)
        idx_va_t = torch.as_tensor(idx_va, device=device, dtype=torch.long)
        X_tr, y_tr = X[idx_tr_t], y_t[idx_tr_t]
        X_va, y_va = X[idx_va_t], y_t[idx_va_t]

        total = 0.0
        for i in range(0, X_tr.size(0), bs):
            xb = X_tr[i:i+bs]
            yb = y_tr[i:i+bs].view(-1)
            pred = model(xb).view(-1)
            loss = loss_fn(pred, yb)

            opt.zero_grad(set_to_none=True)
            loss.backward()
            if grad_clip and grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            opt.step()
            total += float(loss.item()) * xb.size(0)
        train_loss = total / max(1, X_tr.size(0))

        model.eval()
        total = 0.0
        with torch.no_grad():
            for i in range(0, X_va.size(0), bs):
                xb = X_va[i:i+bs]
                yb = y_va[i:i+bs].view(-1)
                pred = model(xb).view(-1)
                loss = loss_fn(pred, yb)
                total += float(loss.item()) * xb.size(0)
        val_loss = total / max(1, X_va.size(0))

        sched.step()
        lr_now = opt.param_groups[0]["lr"]
        print(f"Epoch {epoch:03d}/{epochs} | lr={lr_now:.2e} | train={train_loss:.5f} | val={val_loss:.5f}", flush=True)

        if es_patience and epoch >= es_warmup:
            improved = (best_val - val_loss) > es_min_delta
            if improved:
                best_val = float(val_loss)
                bad_epochs = 0
                if es_restore:
                    best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            else:
                bad_epochs += 1
                if bad_epochs >= es_patience:
                    print(f"[early_stop] epoch={epoch} best_val={best_val:.5f} "
                          f"patience={es_patience} min_delta={es_min_delta}", flush=True)
                    break

    if es_patience and es_restore and best_state is not None:
        model.load_state_dict(best_state, strict=True)

@torch.no_grad()
def sanity_corr_bfs(model, graph, rw_length: int, width: int = 2000, nbt_history_depth: int = 1) -> float:
    model.eval()
    X_s, y_s = graph.random_walks(width=width, length=rw_length, mode="bfs", nbt_history_depth=nbt_history_depth)
    pred = model(X_s.long()).float().view(-1).detach().cpu().numpy()
    y_raw = y_s.float().view(-1).detach().cpu().numpy()
    y_t = np.log1p(y_raw)
    return float(np.corrcoef(pred, y_t)[0, 1])

Идея: использовать обученную модель как эвристику для поиска. Модель выдаёт оценку “насколько состояние далеко от цели”, и эта оценка используется внутри beam search для ранжирования кандидатов.

В отличие от чисто ручной эвристики, здесь ML-оценка считается батчами и комбинируется с лёгкой структурной метрикой gap_h для стабилизации.

Функция beam_improve_with_ml пытается улучшить baseline-решение через beam search, используя ML-эвристику h_fn как основное ранжирование кандидатов. Длина baseline (best_len) задаёт верхнюю границу: все пути, которые уже не могут стать лучше baseline, отсекаются. Возвращается улучшенный путь, если найден, иначе — baseline, что гарантирует “не хуже baseline” в нормальном режиме работы

Дополнительно предусмотрены утилиты для формирования подвыборок тестовых примеров фиксированного размера n и для оценки выигрыша ML-поиска относительно baseline.

In [ ]:
class MLHeuristic:
    """Returns float score per state. Smaller = better."""
    def __init__(self, model, device, batch_size: int = 8192):
        self.model = model
        self.device = device
        self.bs = int(batch_size)

    @torch.no_grad()
    def __call__(self, states):
        self.model.eval()
        out = np.empty(len(states), dtype=np.float32)
        i = 0
        while i < len(states):
            j = min(i + self.bs, len(states))
            x = torch.as_tensor(states[i:j], device=self.device, dtype=torch.long)
            y = self.model(x).float().view(-1)
            out[i:j] = y.detach().cpu().numpy().astype(np.float32)
            i = j
        return out

def beam_improve_with_ml(
    perm,
    h_fn,
    *,
    baseline_moves_fn,
    beam_width=256,
    depth=192,
    w=0.5,
    w_gap=0.15,
    gap_mode="log1p",
    patience=None,
):
    start = list(perm)
    base_moves = baseline_moves_fn(start)
    best_len = len(base_moves)
    if best_len == 0:
        return []

    n = len(start)
    k_values = range(2, n + 1)

    beam = [(0.0, 0, start, [])]  # (f, g, state, path)
    best_path = None
    best_g = {tuple(start): 0}
    no_improve_steps = 0

    for step in range(depth):
        cand_states = []
        cand_meta = []

        for f, g, state, path in beam:
            if g >= best_len:
                continue
            for k in k_values:
                new_g = g + 1
                if new_g >= best_len:
                    continue

                nxt = apply_move_copy(state, k)
                key = tuple(nxt)

                prev = best_g.get(key)
                if prev is not None and prev <= new_g:
                    continue
                best_g[key] = new_g

                new_path = path + [k]

                if is_solved(nxt):
                    best_len = new_g
                    best_path = new_path
                    no_improve_steps = 0
                    continue

                cand_states.append(nxt)
                cand_meta.append((new_g, nxt, new_path))

        if not cand_states:
            break

        h_ml = h_fn(cand_states).astype(np.float32, copy=False)

        gvals = np.fromiter((gap_h(s) for s in cand_states), dtype=np.float32, count=len(cand_states))
        if gap_mode == "log1p":
            gvals = np.log1p(gvals)
        elif gap_mode == "norm":
            gvals = gvals / max(1.0, float(n))
        elif gap_mode == "none":
            pass
        else:
            raise ValueError("gap_mode must be: none | log1p | norm")

        h = w * h_ml + w_gap * gvals

        candidates = [(new_g + float(hh), new_g, nxt, new_path)
                      for (new_g, nxt, new_path), hh in zip(cand_meta, h)]
        beam = nsmallest(beam_width, candidates, key=lambda x: x[0])

        no_improve_steps += 1
        if patience is not None and no_improve_steps >= patience:
            break

    return best_path if best_path is not None else base_moves

def build_rows_for_n(test_df, target_n: int, k: int = 50, seed: int = 42):
    if "n" not in test_df.columns:
        tmp = test_df.copy()
        tmp["n"] = tmp["permutation"].apply(lambda x: len(parse_permutation(x)))
        sub = tmp[tmp["n"] == target_n].reset_index(drop=True)
    else:
        sub = test_df[test_df["n"] == target_n].reset_index(drop=True)

    if len(sub) == 0:
        raise ValueError(f"No rows with n={target_n}")

    rng = np.random.RandomState(seed)
    idx = rng.choice(len(sub), size=min(k, len(sub)), replace=False)

    rows = []
    for i in idx:
        rid = int(sub.loc[i, "id"])
        perm = parse_permutation(sub.loc[i, "permutation"])
        rows.append((rid, perm))
    return rows

def eval_ml_on_rows(
    rows,
    *,
    h_ml,
    baseline_moves_fn,
    beam_width=256,
    depth=192,
    w=0.5,
    w_gap=0.15,
    gap_mode="log1p",
    patience=None,
    log_every=10,
):
    t0 = time.time()
    sum_base = 0
    sum_ml = 0
    improved = same = worse = 0
    total_gain_pos = 0
    max_gain = 0
    max_gain_id = None
    N = len(rows)

    for i, (rid, perm) in enumerate(rows, start=1):
        base = baseline_moves_fn(perm)
        lb = len(base)

        ml_moves = beam_improve_with_ml(
            perm, h_fn=h_ml,
            baseline_moves_fn=baseline_moves_fn,
            beam_width=beam_width, depth=depth, w=w,
            w_gap=w_gap, gap_mode=gap_mode,
            patience=patience,
        )
        lm = len(ml_moves)

        sum_base += lb
        sum_ml += lm

        gain = lb - lm
        if gain > 0:
            improved += 1
            total_gain_pos += gain
            if gain > max_gain:
                max_gain = gain
                max_gain_id = rid
        elif gain == 0:
            same += 1
        else:
            worse += 1

        if log_every and (i % log_every == 0 or i == 1 or i == N):
            print(f"  [{i:4d}/{N}] base={lb:3d} ml={lm:3d} gain={gain:3d}  dt={time.time()-t0:7.1f}s", flush=True)

    dt = time.time() - t0
    return {
        "baseline_total": int(sum_base),
        "ml_total": int(sum_ml),
        "total_gain": int(sum_base - sum_ml),
        "improved_cases": int(improved),
        "same_cases": int(same),
        "worse_cases": int(worse),
        "avg_gain_when_improved": float(total_gain_pos / improved) if improved else 0.0,
        "max_gain": int(max_gain),
        "max_gain_id": int(max_gain_id) if max_gain_id is not None else None,
        "time_sec_eval": float(dt),
        "sec_per_sample_eval": float(dt / max(1, N)),
        "mean_baseline_len": float(sum_base / max(1, N)),
        "mean_ml_len": float(sum_ml / max(1, N)),
        "improved_frac": float(improved / max(1, N)),
    }


Реализуем функцию для проведения экспериментов с прогоном по нескольким размерам n и сетке архитектур model_grid. Для каждого n строится pancake-граф, формируется фиксированный набор тест-кейсов, затем обучается модель на сгенерированных random-walk данных и используется как эвристика внутри beam search.


Мы оцениваем модель по конечному выигрышу в длине решения при использовании её как эвристики в beam search.

In [ ]:
def build_pancake_graph(target_n: int, device: str):
    central_state = list(range(target_n))
    graph = CayleyGraph(
        PermutationGroups.pancake(target_n).make_inverse_closed().with_central_state(central_state),
        device=device,
        dtype=torch.int8,
        batch_size=2**16,
    )
    return graph

def run_one_experiment_cached(
    *,
    rows,
    graph,
    target_n: int,
    cfg: dict,
    exp_name: str,
    out_dir: str,
    baseline_moves_fn,
    beam_width=256,
    depth=192,
    w=0.5,
    w_gap=0.15,
    gap_mode="log1p",
    patience=None,
):
    set_seed(int(cfg.get("seed", 123)))

    CFG = dict(cfg)
    CFG["n"] = target_n
    CFG["num_classes"] = target_n
    CFG["state_size"] = target_n

    k = int(CFG.get("k", 4))
    rw_length_add = int(CFG.get("rw_length_add", 30))
    CFG["rw_length"] = target_n * (target_n + 5) // (4 * (k - 1)) + rw_length_add

    model = get_model(CFG).to(graph.device)

    print(f"\n[{now_str()}] EXP={exp_name} | model={CFG.get('model_type')} n={target_n} "
          f"| epochs={CFG['num_epochs']} rw_width={CFG['rw_width']} rw_mode={CFG.get('rw_mode')}", flush=True)

    t_train0 = time.time()
    train_model_gpu(CFG, model, graph)
    train_time = time.time() - t_train0

    try:
        corr = sanity_corr_bfs(
            model, graph,
            rw_length=CFG["rw_length"],
            width=int(CFG.get("sanity_width", 2000)),
            nbt_history_depth=int(CFG.get("nbt_history_depth", 1)),
        )
    except Exception as e:
        corr = float("nan")
        print("[sanity] failed:", repr(e), flush=True)

    h_ml = MLHeuristic(model, device=graph.device, batch_size=int(CFG.get("h_batch_size", 8192)))

    eval_stats = eval_ml_on_rows(
        rows,
        h_ml=h_ml,
        baseline_moves_fn=baseline_moves_fn,
        beam_width=beam_width, depth=depth, w=w,
        w_gap=w_gap, gap_mode=gap_mode,
        patience=patience,
        log_every=int(CFG.get("eval_log_every", 10)),
    )

    res = {
        "exp_name": exp_name,
        "timestamp": now_str(),
        "target_n": int(target_n),
        "rows_k": int(len(rows)),

        "model_type": str(CFG.get("model_type")),
        "emb_dim": int(CFG["emb_dim"]) if CFG.get("model_type") == "EmbMLP" else None,
        "use_pos_emb": bool(CFG.get("use_pos_emb", True)) if CFG.get("model_type") == "EmbMLP" else None,
        "hd1": int(CFG.get("hd1", 0)),
        "hd2": int(CFG.get("hd2", 0)),
        "nrd": int(CFG.get("nrd", 0)),
        "dropout_rate": float(CFG.get("dropout_rate", 0.0)),

        "rw_width": int(CFG.get("rw_width", 0)),
        "rw_length": int(CFG.get("rw_length", 0)),
        "rw_mode": str(CFG.get("rw_mode", "")),
        "mix_bfs_frac": float(CFG.get("mix_bfs_frac", 0.0)),
        "nbt_history_depth": int(CFG.get("nbt_history_depth", 1)),
        "lr": float(CFG.get("lr", 0.0)),
        "weight_decay": float(CFG.get("weight_decay", 0.0)),
        "batch_size": int(CFG.get("batch_size", 0)),
        "val_ratio": float(CFG.get("val_ratio", 0.0)),
        "num_epochs": int(CFG.get("num_epochs", 0)),
        "y_transform": str(CFG.get("y_transform", "")),
        "loss_beta": float(CFG.get("loss_beta", 1.0)),
        "grad_clip": float(CFG.get("grad_clip", 0.0)),

        "beam_width": int(beam_width),
        "depth": int(depth),
        "w": float(w),
        "w_gap": float(w_gap),
        "gap_mode": str(gap_mode),
        "patience": patience if patience is None else int(patience),

        "train_time_sec": float(train_time),
        "sanity_corr": float(corr),
        **eval_stats,
    }

    ensure_dir(out_dir)
    with open(os.path.join(out_dir, f"{exp_name}.json"), "w") as f:
        json.dump(res, f, indent=2)

    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return res

def run_sweep_n_list(
    *,
    test_df: pd.DataFrame,
    n_list: list[int],
    rows_k: int = 50,
    rows_seed: int = 42,
    model_grid: list[dict],
    w_gap: float = 0.15,
    gap_mode: str = "log1p",
    results_csv: str = "ml_sweep/results_nlist.csv",
    out_json_dir: str = "ml_sweep/json",
    device: str | None = None,
    beam_width: int = 256,
    depth: int = 192,
    w: float = 0.5,
):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    ensure_dir(out_json_dir)
    ensure_dir(os.path.dirname(results_csv))

    done = set()
    if os.path.exists(results_csv):
        prev = pd.read_csv(results_csv, usecols=["exp_name"])
        done = set(prev["exp_name"].astype(str))
        print(f"[resume] {len(done)} experiments already done", flush=True)

    rows_cache = {n: build_rows_for_n(test_df, n, k=rows_k, seed=rows_seed) for n in n_list}
    graph_cache = {n: build_pancake_graph(n, device=device) for n in n_list}

    total = len(n_list) * len(model_grid)
    run_i = 0

    try:
        for n in n_list:
            print(f"\n########## TARGET_N = {n} ##########", flush=True)
            graph = graph_cache[n]
            rows = rows_cache[n]

            best_gain = -10**9
            best_name = None

            for cfg in model_grid:
                run_i += 1
                exp_name = exp_name_from(cfg, n=n, w_gap=w_gap, gap_mode=gap_mode)

                if exp_name in done:
                    print(f"[skip] {run_i}/{total} {exp_name}", flush=True)
                    continue

                print(f"\n===== [{run_i}/{total}] {exp_name} =====", flush=True)

                res = run_one_experiment_cached(
                    rows=rows,
                    graph=graph,
                    target_n=n,
                    cfg=cfg,
                    exp_name=exp_name,
                    out_dir=out_json_dir,
                    baseline_moves_fn=pancake_sort_moves,
                    beam_width=beam_width,
                    depth=depth,
                    w=w,
                    w_gap=w_gap,
                    gap_mode=gap_mode,
                    patience=None,
                )

                df1 = pd.DataFrame([res])
                header = not os.path.exists(results_csv)
                df1.to_csv(results_csv, mode="a", header=header, index=False)

                done.add(exp_name)

                if res["total_gain"] > best_gain:
                    best_gain = res["total_gain"]
                    best_name = exp_name

                print(f"[best@n={n}] total_gain={best_gain} ({best_name})", flush=True)

    finally:
        for g in graph_cache.values():
            try:
                g.free_memory()
            except Exception:
                pass
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    return pd.read_csv(results_csv)


Зададим список n, на которых будет проводиться ML-оценка. Эти значения представляют малый, средний и более сложный режимы, позволяя увидеть, как масштабирование влияет на качество эвристики.

Для каждого n сравниваются разные архитектуры (embedding-модели и one-hot MLP) по их способности улучшать baseline-решение в beam search. Варьируем размер эмбеддингов, наличие позициционных эмбеддингов и количество residual блоков.

In [ ]:
n_list = [20, 30, 50]

model_grid = build_model_grid(
    emb_dims=(32, 64),
    pos_opts=(True, False),
    nrds=(0, 2, 6),
    include_mlpres1=True,
)

df = run_sweep_n_list(
    test_df=test_df,
    n_list=n_list,
    rows_k=20,
    rows_seed=42,
    model_grid=model_grid,
    w_gap=0.15,
    gap_mode="log1p",
    results_csv="ml_sweep/result_n20_30_50.csv",
    out_json_dir="ml_sweep/json",
    beam_width=256,
    depth=192,
    w=0.5,
)

Представление перестановки через embedding элементов (и опционально позиций) существенно лучше one-hot кодирования для этой задачи.

Positional embedding не обязателен на малых n. Увеличение nrd с 2 до 6 не даёт выигрыша, но увеличивает время обучения. emb_dim=64 слегка лучше 32, но разница минимальна.

при росте n модель начинает выигрывать от информации о позиции элемента, а не только от его значения.

На больших n требуется большая глубина модели (nrd=6), emb_dim=32 оказывается лучше 64 (интересный эффект регуляризации). Positional embedding уже не критичен, а иногда даже ухудшает результат. Цена выигрыша - очень серьёзное время поиска.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/pancake_runs/result_n20_30_50.csv")

best_per_n = (
    df.sort_values(["target_n", "total_gain", "sanity_corr"], ascending=[True, False, False])
      .groupby("target_n", as_index=False)
      .head(5)
)
display(best_per_n[["target_n","exp_name","model_type","emb_dim","use_pos_emb","nrd","total_gain","mean_ml_len","sanity_corr","train_time_sec","time_sec_eval"]])

,target_n,exp_name,model_type,emb_dim,use_pos_emb,nrd,total_gain,mean_ml_len,sanity_corr,train_time_sec,time_sec_eval
11,20,n20_EmbMLP_ed64_pos0_nrd6_wg0.15_log1p,EmbMLP,64.0,False,6,229,18.70,0.924308,53.385489,18.869771
10,20,n20_EmbMLP_ed64_pos0_nrd2_wg0.15_log1p,EmbMLP,64.0,False,2,229,18.70,0.923873,34.667781,17.854649
7,20,n20_EmbMLP_ed64_pos1_nrd2_wg0.15_log1p,EmbMLP,64.0,True,2,228,18.75,0.927537,35.715142,18.047640
4,20,n20_EmbMLP_ed32_pos0_nrd2_wg0.15_log1p,EmbMLP,32.0,False,2,228,18.75,0.922724,34.754986,17.334467
8,20,n20_EmbMLP_ed64_pos1_nrd6_wg0.15_log1p,EmbMLP,64.0,True,6,227,18.80,0.924568,54.465402,18.901072
22,30,n30_EmbMLP_ed64_pos1_nrd2_wg0.15_log1p,EmbMLP,64.0,True,2,386,29.90,0.953272,64.946194,56.314097
25,30,n30_EmbMLP_ed64_pos0_nrd2_wg0.15_log1p,EmbMLP,64.0,False,2,374,30.50,0.950571,63.166497,58.021820
23,30,n30_EmbMLP_ed64_pos1_nrd6_wg0.15_log1p,EmbMLP,64.0,True,6,369,30.75,0.948167,95.269771,61.951936
16,30,n30_EmbMLP_ed32_pos1_nrd2_wg0.15_log1p,EmbMLP,32.0,True,2,368,30.80,0.948684,63.761750,57.413650
19,30,n30_EmbMLP_ed32_pos0_nrd2_wg0.15_log1p,EmbMLP,32.0,False,2,367,30.85,0.949092,61.264268,57.142353


One-hot MLP существенно уступает embedding-подходу для задачи pancake-графа, даже при увеличении глубины.
Однако глубина (residual stack)также критична для качества ML-эвристики.

emb_dim=32 стабильно лучше, чем 64.
Более компактное представление элемента оказывается эффективнее для эвристического поиска.

positional embedding не является определяющим фактором

In [ ]:
agg = (
    df.groupby(["model_type","emb_dim","use_pos_emb","nrd"], dropna=False, as_index=False)
      .agg(total_gain_sum=("total_gain","sum"),
           mean_ml_len_mean=("mean_ml_len","mean"),
           sanity_corr_mean=("sanity_corr","mean"))
      .sort_values(["total_gain_sum","sanity_corr_mean"], ascending=[False, False])
)
display(agg.head(20))


,model_type,emb_dim,use_pos_emb,nrd,total_gain_sum,mean_ml_len_mean,sanity_corr_mean
2,EmbMLP,32.0,False,6,997,38.816667,0.939317
5,EmbMLP,32.0,True,6,978,39.133333,0.938082
11,EmbMLP,64.0,True,6,951,39.583333,0.944556
8,EmbMLP,64.0,False,6,948,39.633333,0.944285
4,EmbMLP,32.0,True,2,918,40.133333,0.941958
1,EmbMLP,32.0,False,2,900,40.433333,0.942980
7,EmbMLP,64.0,False,2,886,40.666667,0.943780
10,EmbMLP,64.0,True,2,861,41.083333,0.945967
14,MLPRes1,NaN,NaN,6,686,44.000000,0.920993
13,MLPRes1,NaN,NaN,2,596,45.500000,0.926183


Лучшей архитектурой в среднем по всем протестированным размерам задач является:

**EmbMLP с emb_dim = 32 и nrd = 6**

При этом использование positional embedding остаётся опциональным.

Эта модель демонстрирует оптимальный баланс между выразительностью, обобщающей способностью и практической эффективностью в задаче эвристического поиска в pancake-графе.

Приведенная ниже функция формирует строковый идентификатор эксперимента на основе конфигурации модели, размера задачи n и параметров эвристического поиска. Для embedding-моделей в имя включаются размер эмбеддинга, наличие positional embedding и глубина residual-стека, а для остальных моделей — только тип и глубина.

Дополнительно кодируются параметры генерации данных (rw_width, mix_bfs_frac), обучения (learning rate, weight decay, число эпох) и параметры эвристики (w_gap, gap_mode), что гарантирует уникальность и воспроизводимость экспериментов.

In [ ]:
def exp_name_from(cfg: dict, *, n: int, w_gap: float, gap_mode: str) -> str:

    mt = cfg.get("model_type", "EmbMLP")

    if mt == "EmbMLP":
        ed  = cfg.get("emb_dim", 0)
        pos = int(bool(cfg.get("use_pos_emb", True)))
        nrd = int(cfg.get("nrd", 0))
        tag = f"{mt}_ed{ed}_pos{pos}_nrd{nrd}"
    else:
        nrd = int(cfg.get("nrd", 0))
        tag = f"{mt}_nrd{nrd}"

    rw = int(cfg.get("rw_width", 0))
    mf = cfg.get("mix_bfs_frac", None)
    mf_tag = f"_mix{float(mf):.2f}" if mf is not None else ""
    lr = cfg.get("lr", None)
    wd = cfg.get("weight_decay", None)
    ep = int(cfg.get("num_epochs", 0))

    return f"n{n}_{tag}_rw{rw}{mf_tag}_lr{lr}_wd{wd}_ep{ep}_wg{w_gap}_{gap_mode}"


Для более тонкой настройки мы фиксируем лучшую по прошлым экспериментам архитектуру (EmbMLP с emb_dim=32 и nrd=6) и перебираем только параметры обучения и генерации данных, чтобы улучшить качество ML-эвристики. Перебор запускается на одном выбранном размере n (здесь n=30) как компромиссе между информативностью и вычислительной стоимостью.

В этом режиме различия между конфигурациями обучения наиболее отчётливо проявляются в итоговом выигрыше поиска, при этом вычислительная стоимость остаётся приемлемой.

Рассматриваются следующие параметры:
rw_width (регулирует объём сгенерированных данных на эпоху), mix_frac_grid (доля BFS-примеров в смешанном датасете), а lr и weight_decay (поведение оптимизатора и регуляризацию).

In [ ]:
LEADER_ARCH = dict(
    model_type="EmbMLP",
    emb_dim=32,
    use_pos_emb=False,
    hd1=512, hd2=256, nrd=6,
    dropout_rate=0.1,
)

TRAIN_BASE = dict(
    rw_mode="mix",
    nbt_history_depth=1,
    k=4,
    rw_length_add=30,

    y_transform="log1p",
    loss_beta=1.0,
    grad_clip=1.0,

    batch_size=1024,
    val_ratio=0.15,

    stratify_clip=60,
    stratify_bin_size=1.0,

    sanity_width=1200,
    h_batch_size=8192,
    eval_log_every=10,

    seed=123,
)

rw_width_grid    = [2000, 3000, 5000]
mix_frac_grid    = [0.15, 0.30, 0.50]
lr_grid          = [1e-4, 2e-4]
weight_decay_grid= [5e-3, 1e-2]

epochs_grid      = [40]
model_grid = []
for rw, mf, lr, wd, ep in product(rw_width_grid, mix_frac_grid, lr_grid, weight_decay_grid, epochs_grid):
    cfg = {}
    cfg.update(LEADER_ARCH)
    cfg.update(TRAIN_BASE)
    cfg.update(dict(
        rw_width=int(rw),
        mix_bfs_frac=float(mf),
        lr=float(lr),
        weight_decay=float(wd),
        num_epochs=int(ep),
    ))
    model_grid.append(cfg)

print("Total training configs:", len(model_grid))

BEAM_BW = 256
BEAM_DEPTH = 192
BEAM_W = 0.5
N_LIST = [30]
ROWS_K = 20
ROWS_SEED = 42

OUT_JSON_DIR = "leader_tune/json"
RESULTS_CSV  = "leader_tune/results_train.csv"

df_train = run_sweep_n_list(
    test_df=test_df,
    n_list=N_LIST,
    rows_k=ROWS_K,
    rows_seed=ROWS_SEED,
    model_grid=model_grid,
    w_gap=0.15,
    gap_mode="log1p",
    results_csv=RESULTS_CSV,
    out_json_dir=OUT_JSON_DIR,
    device=None,   # auto cuda/cpu
    beam_width=BEAM_BW,
    depth=BEAM_DEPTH,
    w=BEAM_W,
)


Total training configs: 36
[resume] 3 experiments already done

########## TARGET_N = 30 ##########

===== [1/36] n30_EmbMLP_ed32_pos0_nrd6_rw2000_mix0.15_lr0.0001_wd0.005_ep40_wg0.15_log1p =====

[2026-02-04 16:52:31] EXP=n30_EmbMLP_ed32_pos0_nrd6_rw2000_mix0.15_lr0.0001_wd0.005_ep40_wg0.15_log1p | model=EmbMLP n=30 | epochs=40 rw_width=2000 rw_mode=mix
Epoch 000/40 | lr=9.98e-05 | train=0.40582 | val=0.16431
Epoch 001/40 | lr=9.94e-05 | train=0.17577 | val=0.14244
Epoch 002/40 | lr=9.86e-05 | train=0.13400 | val=0.10224
Epoch 003/40 | lr=9.76e-05 | train=0.10600 | val=0.07638
Epoch 004/40 | lr=9.62e-05 | train=0.08999 | val=0.06478
Epoch 005/40 | lr=9.46e-05 | train=0.07892 | val=0.05690
Epoch 006/40 | lr=9.26e-05 | train=0.06621 | val=0.05368
Epoch 007/40 | lr=9.05e-05 | train=0.06195 | val=0.04514
Epoch 008/40 | lr=8.80e-05 | train=0.06069 | val=0.04640
Epoch 009/40 | lr=8.54e-05 | train=0.05650 | val=0.04236
Epoch 010/40 | lr=8.25e-05 | train=0.05341 | val=0.04456
Epoch 011/40 | l

,exp_name,timestamp,target_n,rows_k,model_type,emb_dim,use_pos_emb,hd1,hd2,nrd,...,same_cases,worse_cases,avg_gain_when_improved,max_gain,max_gain_id,time_sec_eval,sec_per_sample_eval,mean_baseline_len,mean_ml_len,improved_frac
34,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.30_lr0.0...,2026-02-04 18:48:18,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.6,28,1157,69.792297,3.489615,49.2,29.6,1.0
29,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.15_lr0.0...,2026-02-04 18:23:51,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.6,28,1157,69.195672,3.459784,49.2,29.6,1.0
37,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,2026-02-04 19:03:07,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.5,28,1157,69.276130,3.463806,49.2,29.7,1.0
36,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,2026-02-04 18:58:12,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.4,27,1157,70.756788,3.537839,49.2,29.8,1.0
30,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.15_lr0.0...,2026-02-04 18:28:40,30,20,EmbMLP,32,False,512,256,6,...,0,0,19.4,27,1157,68.937713,3.446886,49.2,29.8,1.0


При фиксированной архитектуре EmbMLP (emb_dim=32, nrd=6) наибольшее влияние на качество эвристики оказывают параметры генерации обучающих данных.
Увеличение ширины случайных обходов улучшает итоговый выигрыш.

Оптимальным оказался режим смешивания BFS и NBT с долей BFS около 30%, а также более агрессивные параметры оптимизации (lr=2e-4, weight decay=1e-2).


In [ ]:
best_gain = (
    df_train.sort_values(["total_gain","sanity_corr"], ascending=[True,False,False]).groupby("total_gain").head(39)
)
display(best_gain[["total_gain","exp_name","rw_width", "lr","weight_decay","mix_bfs_frac","sanity_corr","train_time_sec","time_sec_eval"]])

,total_gain,exp_name,rw_width,lr,weight_decay,mix_bfs_frac,sanity_corr,train_time_sec,time_sec_eval
34,392,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.30_lr0.0...,5000,0.0002,0.010,0.30,0.955797,225.533947,69.792297
29,392,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.15_lr0.0...,5000,0.0002,0.005,0.15,0.949510,223.916882,69.195672
37,390,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,5000,0.0002,0.005,0.50,0.958488,225.368810,69.276130
36,388,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,5000,0.0001,0.010,0.50,0.958075,224.477674,70.756788
30,388,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.15_lr0.0...,5000,0.0002,0.010,0.15,0.950487,219.868918,68.937713
38,387,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,5000,0.0002,0.010,0.50,0.958318,222.863929,69.362046
22,386,n30_EmbMLP_ed32_pos0_nrd6_rw3000_mix0.30_lr0.0...,3000,0.0002,0.010,0.30,0.955731,143.221617,70.109416
26,386,n30_EmbMLP_ed32_pos0_nrd6_rw3000_mix0.50_lr0.0...,3000,0.0002,0.010,0.50,0.955653,142.869051,69.949446
17,386,n30_EmbMLP_ed32_pos0_nrd6_rw3000_mix0.15_lr0.0...,3000,0.0002,0.005,0.15,0.954151,140.495360,69.854921
35,385,n30_EmbMLP_ed32_pos0_nrd6_rw5000_mix0.50_lr0.0...,5000,0.0001,0.005,0.50,0.958345,226.512267,70.597412


Рассчитаем и сохраним базовое решение(baseline) для всех тестовых перестановок, а также реализуем механизм продолжения работы через файл прогресса. Если прогресс уже существует, код подхватывает ранее посчитанные ответы и не пересчитывает их заново. Если прогресса нет, строится baseline_map для всех id в test_df и сохраняется на диск, чтобы дальше отталкиваться от него при улучшениях.

In [ ]:
baseline_moves_fn = pancake_sort_moves

def _save_progress(progress_map: dict, path: str):
    df = pd.DataFrame(list(progress_map.items()), columns=["id", "solution"]).sort_values("id")
    df.to_csv(path, index=False)
if os.path.exists(PROGRESS_PATH):
    prog_df = pd.read_csv(PROGRESS_PATH)
    progress_map = dict(zip(prog_df["id"].astype(int).values, prog_df["solution"].values))
    print("Resume progress:", len(progress_map))
else:
    progress_map = {}
    print("No progress, start fresh.")

    t0 = time.time()
    baseline_map = {}
    for i in range(len(test_df)):
        pid = int(test_df.loc[i, "id"])
        perm = parse_permutation(test_df.loc[i, "permutation"])
        moves = baseline_moves_fn(perm)
        baseline_map[pid] = moves_to_str(moves)
        if (i + 1) % 2000 == 0 or (i + 1) == len(test_df):
            print(f"  baseline [{i+1}/{len(test_df)}] dt={time.time()-t0:.1f}s", flush=True)
    _save_progress(baseline_map, BASELINE_PATH)
    print("Saved baseline:", BASELINE_PATH)


No progress, start fresh.
  baseline [2000/2405] dt=0.1s
  baseline [2405/2405] dt=0.2s
Saved baseline: /content/drive/MyDrive/pancake_runs/baseline_submission.csv


Зафиксируем "финальную" конфигурацию пайплайна: параметры beam search и параметры обучения/использования ML-эвристики, выбранные по результатам тюнинга.

In [ ]:
BEAM_WIDTH = 256
DEPTH = 192
W = 0.5
W_GAP = 0.15
GAP_MODE = "log1p"
H_BATCH = 8192

LEADER_CFG = dict(
    model_type="EmbMLP",
    emb_dim=32,
    use_pos_emb=False,
    hd1=512, hd2=256, nrd=6, dropout_rate=0.1,

    rw_width=5000,
    rw_mode="mix",
    mix_bfs_frac=0.30,
    nbt_history_depth=1,

    lr=2e-4,
    weight_decay=1e-2,
    batch_size=1024,
    val_ratio=0.15,
    num_epochs=30,

    y_transform="log1p",
    grad_clip=1.0,
    loss_beta=1.0,

    sanity_width=2000,
    h_batch_size=H_BATCH,
    eval_log_every=200,
    seed=123,

    stratify_clip=60,
    stratify_bin_size=1.0,
    k=4,
)


Реализуем финальный прогон: для каждого размера n из списка мы обучаем отдельную ML-эвристику на соответствующем pancake-графе, затем применяем её к всем тестовым кейсам этого размера и улучшаем baseline-решение через beam search.

In [ ]:
flush_every = 200
t_global = time.time()

done_cnt = 0
improved_cnt = 0

for n in n_list:
    print("\n========== TARGET n =", n, "==========", flush=True)

    graph = build_pancake_graph(n, device=device)
    CFG = dict(LEADER_CFG)
    CFG["n"] = n
    CFG["num_classes"] = n
    CFG["state_size"] = n

    k = int(CFG.get("k", 4))
    rw_length_add = int(CFG.get("rw_length_add", 30))
    CFG["rw_length"] = n * (n + 5) // (4 * (k - 1)) + rw_length_add

    model = get_model(CFG).to(graph.device)

    print(f"[{now_str()}] train model for n={n} | epochs={CFG['num_epochs']} rw_width={CFG['rw_width']} mix={CFG['mix_bfs_frac']}", flush=True)
    t_train0 = time.time()
    train_model_gpu(CFG, model, graph)
    print(f"train_time={time.time()-t_train0:.1f}s", flush=True)

    h_ml = MLHeuristic(model, device=graph.device, batch_size=H_BATCH)

    sub = test_df[test_df["n"] == n][["id", "permutation"]].reset_index(drop=True)
    total_n = len(sub)
    print("cases:", total_n, flush=True)

    t0 = time.time()
    for i in range(total_n):
        pid = int(sub.loc[i, "id"])
        base_str = baseline_map.get(pid, "")
        if base_str == "":
            perm0 = parse_permutation(sub.loc[i, "permutation"])
            base_str = moves_to_str(baseline_moves_fn(perm0))
            baseline_map[pid] = base_str

        perm = parse_permutation(sub.loc[i, "permutation"])

        base_len = moves_len(base_str)

        moves = beam_improve_with_ml(
            perm,
            h_fn=h_ml,
            baseline_moves_fn=baseline_moves_fn,
            beam_width=BEAM_WIDTH,
            depth=DEPTH,
            w=W,
            w_gap=W_GAP,
            gap_mode=GAP_MODE,
            patience=None
        )
        new_str = moves_to_str(moves)
        new_len = len(moves)

        if new_len < base_len:
            progress_map[pid] = new_str
            improved_cnt += 1
        else:
            progress_map[pid] = base_str

        done_cnt += 1

        if (i + 1) % flush_every == 0 or (i + 1) == total_n:
            _save_progress(progress_map, PROGRESS_PATH)
            dt = time.time() - t0
            print(f"  [{i+1:5d}/{total_n}] improved={improved_cnt} done_total={done_cnt} "
                  f"dt_n={dt:6.1f}s dt_all={time.time()-t_global:7.1f}s", flush=True)
    try:
        graph.free_memory()
    except Exception:
        pass
    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

final_df = pd.DataFrame(list(progress_map.items()), columns=["id", "solution"]).sort_values("id").reset_index(drop=True)
final_df.to_csv(FINAL_PATH, index=False)
print("\nSaved:", FINAL_PATH, "rows=", len(final_df))
print("Progress saved:", PROGRESS_PATH)


Для задач малого и среднего размера (n < 75) beam search с обученной ML-эвристикой достигает того же качества решений, что и beam search с простой gap-эвристикой. На больших n поиск не увенчался успехом.

In [ ]:
stats = evaluate_submission_vs_baseline(
    test_df=test_df,
    submission_df=final_df,
    baseline_moves_fn=pancake_sort_moves,
    log_every=50,
    save_detailed_path="sub_vs_base_details.csv",
)
stats

[   1/2405] base=  2 sub=  2 gain=  0  elapsed=    0.0s
[  50/2405] base= 19 sub=  8 gain= 11  elapsed=    0.0s
[ 100/2405] base= 20 sub= 12 gain=  8  elapsed=    0.0s
[ 150/2405] base= 16 sub= 11 gain=  5  elapsed=    0.0s
[ 200/2405] base= 15 sub= 11 gain=  4  elapsed=    0.0s
[ 250/2405] base= 19 sub= 15 gain=  4  elapsed=    0.0s
[ 300/2405] base= 20 sub= 14 gain=  6  elapsed=    0.0s
[ 350/2405] base= 20 sub= 15 gain=  5  elapsed=    0.0s
[ 400/2405] base= 21 sub= 14 gain=  7  elapsed=    0.0s
[ 450/2405] base= 21 sub= 13 gain=  8  elapsed=    0.0s
[ 500/2405] base= 25 sub= 15 gain= 10  elapsed=    0.0s
[ 550/2405] base= 25 sub= 15 gain= 10  elapsed=    0.0s
[ 600/2405] base= 21 sub= 12 gain=  9  elapsed=    0.0s
[ 650/2405] base= 32 sub= 20 gain= 12  elapsed=    0.0s
[ 700/2405] base= 34 sub= 19 gain= 15  elapsed=    0.0s
[ 750/2405] base= 32 sub= 19 gain= 13  elapsed=    0.0s
[ 800/2405] base= 33 sub= 19 gain= 14  elapsed=    0.0s
[ 850/2405] base= 42 sub= 24 gain= 18  elapsed= 

{'baseline_total': 158680,
 'submission_total': 89252,
 'total_gain': 69428,
 'improved_cases': 2213,
 'same_cases': 192,
 'worse_cases': 0,
 'avg_gain_when_improved': 31.372797107998192,
 'max_gain': 195,
 'max_gain_id': 2210,
 'time_sec': 0.3336763381958008,
 'sec_per_sample': 0.00013874276016457413,
 'mean_baseline_len': 65.97920997920998,
 'mean_submission_len': 37.11101871101871,
 'improved_frac': 0.9201663201663202}

Создадим функцию, объединяющую несколько сабмитов, как полных, так и частичных (где решения есть только для части id), в один финальный файл.

Сначала выбираем base-файл, затем поверх него применяем partial-файлы, заменяя решение только если оно короче. В конце сохраняем merged submission и печатаем сводку, включая итоговый score (сумму длин) и распределение "победителей" по источникам.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Dict, Optional, Tuple


def merge_submissions_with_partials(
    *,
    base_paths: List[str],
    partial_paths: List[str],
    out_path: str = "submission_final.csv",
    save_source_column: bool = True,
    tie_break: str = "keep_base",
) -> pd.DataFrame:

    base_subs: List[pd.DataFrame] = []
    for p in base_paths:
        df = pd.read_csv(p)
        assert {"id", "solution"}.issubset(df.columns), f"{p} must have columns: id, solution"
        df = df[["id", "solution"]].copy()
        df["id"] = df["id"].astype(int)
        df = df.sort_values("id").reset_index(drop=True)
        df["len"] = df["solution"].map(moves_len)
        df["source"] = str("baseline")
        base_subs.append(df)

    base_ids = base_subs[0]["id"].values
    for i, df in enumerate(base_subs[1:], start=1):
        if len(df) != len(base_subs[0]) or not np.array_equal(df["id"].values, base_ids):
            raise ValueError(f"ID mismatch between {base_paths[0]} and {base_paths[i]}")

    best = base_subs[0][["id", "solution", "len", "source"]].copy()
    for df in base_subs[1:]:
        better = df["len"].values < best["len"].values
        best.loc[better, "solution"] = df.loc[better, "solution"].values
        best.loc[better, "len"] = df.loc[better, "len"].values
        best.loc[better, "source"] = df.loc[better, "source"].values

    best["source"] = best["source"].astype(str)

    id_to_idx: Dict[int, int] = {int(pid): i for i, pid in enumerate(best["id"].values)}

    applied_stats = []
    for p in partial_paths:
        if not Path(p).exists():
            print(f"[WARN] partial not found, skipped: {p}")
            continue

        part = pd.read_csv(p)
        assert {"id", "solution"}.issubset(part.columns), f"{p} must have columns: id, solution"
        part = part[["id", "solution"]].copy()
        part["id"] = part["id"].astype(int)
        part["len"] = part["solution"].map(moves_len)

        replaced = 0
        missing = 0
        stem = Path(p).stem
        src_tag = stem.replace("submission_", "")

        for pid, sol, l in part.itertuples(index=False):
            idx = id_to_idx.get(int(pid))
            if idx is None:
                missing += 1
                continue

            cur_len = int(best.at[idx, "len"])
            if l < cur_len or (tie_break == "prefer_partial" and l == cur_len and sol != best.at[idx, "solution"]):
                best.at[idx, "solution"] = sol
                best.at[idx, "len"] = int(l)
                best.at[idx, "source"] = src_tag
                replaced += 1

        applied_stats.append((str(p), replaced, missing, len(part)))

    out_df = best[["id", "solution"]].copy()
    if save_source_column:
        out_df["source"] = best["source"].copy()

    out_df.to_csv(out_path, index=False)

    total_moves = int(best["len"].sum())
    print("\n=== MERGE SUMMARY ===")
    print("Output:", out_path)
    print("Rows:", len(out_df))
    print("Total moves (score):", total_moves)

    if save_source_column:
        print("\nFinal winners by source tag (top):")
        display(out_df["source"].value_counts().head(20))

    print("\nSaved:", out_path)
    display(out_df.head(10))
    return out_df


В качестве главного сабмита используется один baseline-файл, и он служит опорной точкой, от которой мы будем отталкиваться при улучшениях.

В качестве дополнительных используем результат beam search с gap-эвристикой и результат ML-эвристики, сохранённый по ходу прогресса. Эти файлы не обязаны покрывать весь датасет и применяются поверх baseline только там, где они действительно улучшают решение.

In [ ]:
BASE_PATHS = [
    "/content/drive/MyDrive/pancake_runs/submission_baseline.csv"
]

PARTIAL_PATHS = [
    "/content/drive/MyDrive/pancake_runs/submission_gap.csv",
    "/content/drive/MyDrive/pancake_runs/submission_progress_ml.csv",
]

merged_df = merge_submissions_with_partials(
    base_paths=BASE_PATHS,
    partial_paths=PARTIAL_PATHS,
    out_path="submission_final.csv",
    save_source_column=True,
    tie_break="keep_base",
)



=== MERGE SUMMARY ===
Output: submission_final.csv
Rows: 2405
Total moves (score): 91584

Final winners by source tag (top):


,count
source,
gap,2391
progress_ml,10
baseline,4



Saved: submission_final.csv


,id,solution,source
0,0,R4.R2,baseline
1,1,R5.R4.R2.R4,gap
2,2,R4.R2.R3.R2,baseline
3,3,R2.R3,baseline
4,4,R3.R5.R3.R4,baseline
5,5,R3.R2.R12.R5.R11.R8.R5.R9.R5.R10.R3.R7,gap
6,6,R10.R11.R4.R7.R6.R8.R10.R3.R12.R8,gap
7,7,R11.R6.R12.R4.R5.R3.R10.R3.R11.R5.R8,gap
8,8,R4.R7.R10.R6.R8.R6.R12.R9.R8.R4,gap
9,9,R10.R3.R6.R9.R8.R11.R4.R12.R7.R2.R5,gap


Подавляющее большинство финальных решений (2391 из 2405) пришло из beam search с gap-эвристикой. Это подтверждает, что для основного диапазона задач именно gap-эвристика даёт наиболее стабильные и воспроизводимые улучшения.

Фактически, именно она формирует "скелет" итогового сабмита.

Хотя вклад ML-эвристики количественно небольшой (10 кейсов), он ненулевой: все замены - строгие улучшения, ML действительно находит решения, которые gap-эвристика пропускает.

Финальный сабмит получен путём безопасного объединения baseline-решений с результатами двух улучшенных методов: beam search с gap-эвристикой и beam search с ML-эвристикой. Замены выполнялись только при строгом улучшении, что полностью исключает деградации.

Наилучший результат достигается гибридной стратегией:
beam search с ручной gap-эвристикой как основным методом и ML-эвристикой как дополнительным источником точечных улучшений.


Итоговый score: **91584**

# Про ML-модели

Во всех экспериментах модели класса EmbMLP значительно опережают one-hot MLP (MLPRes1) по суммарному выигрышу в длине решения. Даже при увеличении глубины и сложности one-hot архитектуры остаются заметно слабее embedding-подхода. Это указывает на то, что явное представление элементов перестановки в виде эмбеддингов является более подходящим способом кодирования состояний pancake-графа.

Число residual-блоков (nrd) оказывает наибольшее влияние на итоговый результат. Модели с nrd=6 стабильно демонстрируют наилучшие показатели суммарного выигрыша, тогда как уменьшение глубины приводит к заметной деградации качества. Это говорит о том, что для построения эффективной эвристики требуется достаточно выразительная нелинейная модель, способная захватывать сложные зависимости между элементами перестановки.

Внезапно, модели с размерностью эмбеддинга 32 в среднем превосходят модели с emb_dim=64. Более компактное представление элементов перестановки оказывается менее склонным к переобучению на данных случайных обходов и лучше переносится на задачу эвристического поиска. Этот результат подчёркивает, что увеличение выразительности модели не всегда приводит к улучшению её практической эффективности.

Использование positional embedding даёт лишь незначительный эффект и не является критичным для достижения высоких результатов. В некоторых режимах (средние значения n) оно может слегка улучшать качество, однако в среднем влияние позиционных эмбеддингов оказывается слабым по сравнению с вкладом глубины модели и способа кодирования состояний.

Хотя высокая корреляция предсказаний модели с BFS-дистанциями необходима для успешной эвристики, она не гарантирует максимального выигрыша в beam search. Некоторые модели с более высокой sanity_corr показывают меньший суммарный выигрыш, чем более «простые» архитектуры. Это подтверждает важность оценки моделей по конечному эффекту в поиске, а не только по регрессионным метрикам.